In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats
import statsmodels.formula.api as smf
from sklearn import linear_model
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
L2M = pd.read_csv('/content/drive/Shared drives/DSO 510 Final Project/L2M New.csv')
L2M.head(5)

,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,call,...,committing_side,disadvantaged_min,disadvantaged_team,disadvantaged_side,type2,season,playoff,Favored Team,Have_SuperPlayer_or_not,Num_super_player
0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,home,30.033333,GSW,away,TRAVELING,2015,False,NaN,NaN,NaN
1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,home,NaN,NaN,NaN,OUT OF BOUNDS,2015,False,NaN,NaN,NaN
2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,FOUL,...,away,33.950000,DEN,home,LOOSE BALL,2015,False,NaN,NaN,NaN
3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,away,42.350000,HOU,home,DOUBLE PERSONAL,2015,False,NaN,NaN,NaN
4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,home,42.100000,CLE,away,OFFENSIVE,2015,False,NaN,NaN,NaN


In [ ]:
# Replace the nulls in disadvantaged_team columns
# If committing_team is home team, disadvantaged_team is away team.
L2M.loc[L2M['committing_team'] == L2M['home_team'],'disadvantaged_team'] = L2M.loc[L2M['committing_team'] == L2M['home_team'],'away_team']
L2M.loc[L2M['committing_team'] == L2M['away_team'],'disadvantaged_team'] = L2M.loc[L2M['committing_team'] == L2M['away_team'],'home_team']

In [ ]:
# Favored Team in each call
# If the call is INC, favored team is committing_team
# If the call is IC, favored team is disadvantaged_team
L2M.loc[L2M['decision']== 'INC','Favored Team'] = L2M.loc[L2M['decision']== 'INC', 'committing_team']
L2M.loc[L2M['decision']== 'IC','Favored Team'] = L2M.loc[L2M['decision']== 'IC', 'disadvantaged_team']

In [ ]:
L2M.head(5)

,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,call,...,committing_side,disadvantaged_min,disadvantaged_team,disadvantaged_side,type2,season,playoff,Favored Team,Have_SuperPlayer_or_not,Num_super_player
0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,home,30.033333,GSW,away,TRAVELING,2015,False,BOS,NaN,NaN
1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,home,NaN,GSW,NaN,OUT OF BOUNDS,2015,False,BOS,NaN,NaN
2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,FOUL,...,away,33.950000,DEN,home,LOOSE BALL,2015,False,NOP,NaN,NaN
3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,away,42.350000,HOU,home,DOUBLE PERSONAL,2015,False,CLE,NaN,NaN
4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,home,42.100000,CLE,away,OFFENSIVE,2015,False,HOU,NaN,NaN


In [ ]:
L2M['Favored Team'].isnull().sum()

29

In [ ]:
Super_player = pd.read_csv('/content/drive/Shared drives/DSO 510 Final Project/Super players.csv')

Super_player.head(3)

,Team,Player1,Player2,Player3,Counts,Year
0,CLE,LeBron James,Kyrie Irving,NaN,2,2015
1,GSW,Stephen Curry,NaN,NaN,1,2015
2,OKC,Kevin Durant,NaN,NaN,1,2015


In [ ]:
# Set Have_SuperPlayer_or_not, Num_super_player to 0
L2M.loc[:,['Have_SuperPlayer_or_not','Num_super_player']] = 0

<ipython-input-9-b83cf33b6d7d>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  L2M.loc[:,['Have_SuperPlayer_or_not','Num_super_player']] = 0


In [ ]:
# L2M['season'] == super_players['year'] & L2M['Favored Team'] == super_players['Team'] -> L2M['Num_super_players'] = super_players['Counts']
merged_df = L2M.merge(Super_player, left_on=['season', 'Favored Team'], right_on=['Year', 'Team'], how='left')

In [ ]:
merged_df.head(3)

,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,call,...,playoff,Favored Team,Have_SuperPlayer_or_not,Num_super_player,Team,Player1,Player2,Player3,Counts,Year
0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,False,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,False,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,FOUL,...,False,NOP,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# How many Super players in the favor team

L2M['Num_super_player'] = merged_df['Counts']
L2M['Num_super_player'].fillna(0, inplace=True)

In [ ]:
# Does the team has at least one super player

L2M.loc[L2M['Num_super_player'] != 0,'Have_SuperPlayer_or_not'] = 1

In [ ]:
L2M.head(5)

,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,call,...,committing_side,disadvantaged_min,disadvantaged_team,disadvantaged_side,type2,season,playoff,Favored Team,Have_SuperPlayer_or_not,Num_super_player
0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,home,30.033333,GSW,away,TRAVELING,2015,False,BOS,0,0.0
1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,home,NaN,GSW,NaN,OUT OF BOUNDS,2015,False,BOS,0,0.0
2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,FOUL,...,away,33.950000,DEN,home,LOOSE BALL,2015,False,NOP,0,0.0
3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,away,42.350000,HOU,home,DOUBLE PERSONAL,2015,False,CLE,1,2.0
4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,home,42.100000,CLE,away,OFFENSIVE,2015,False,HOU,0,0.0


In [ ]:
L2M.rename(columns ={'Have_SuperPlayer_or_not':'FT_is_ST'}, inplace = True)
L2M.rename(columns ={'Num_super_player':'Num_SP_in_FT'}, inplace = True)

In [ ]:
# Assign a new variable called non-favored team in each game
L2M.loc[L2M['decision']== 'INC','non Favored Team'] = L2M.loc[L2M['decision']== 'INC', 'disadvantaged_team']
L2M.loc[L2M['decision']== 'IC','non Favored Team'] = L2M.loc[L2M['decision']== 'IC', 'committing_team']

In [ ]:
merged_df2 = L2M.merge(Super_player, left_on=['season', 'non Favored Team'], right_on=['Year', 'Team'], how='left')
merged_df2.head(5)

,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,call,...,Favored Team,FT_is_ST,Num_SP_in_FT,non Favored Team,Team,Player1,Player2,Player3,Counts,Year
0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,BOS,0,0.0,GSW,GSW,Stephen Curry,NaN,NaN,1.0,2015.0
1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,BOS,0,0.0,GSW,GSW,Stephen Curry,NaN,NaN,1.0,2015.0
2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,FOUL,...,NOP,0,0.0,DEN,NaN,NaN,NaN,NaN,NaN,NaN
3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,CLE,1,2.0,HOU,NaN,NaN,NaN,NaN,NaN,NaN
4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,HOU,0,0.0,CLE,CLE,LeBron James,Kyrie Irving,NaN,2.0,2015.0


In [ ]:
L2M['Num_SP_in_NFT'] = merged_df2['Counts']
L2M['Num_SP_in_NFT'].fillna(0, inplace=True)

In [ ]:
L2M.loc[L2M['Num_SP_in_NFT'] != 0,'NFT_is_ST'] = 1
L2M['NFT_is_ST'].fillna(0, inplace=True)

In [ ]:
L2M['ST_in_call']= L2M['FT_is_ST'] + L2M['NFT_is_ST']

In [ ]:
L2M.dropna(subset = ['Favored Team'], inplace = True)

In [ ]:
L2M.head(5)
L2M.shape

(5320, 35)

In [ ]:
L2M['Favored Team'].isnull().sum()

0

In [ ]:
L2M.head(5)

,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,call,...,type2,season,playoff,Favored Team,FT_is_ST,Num_SP_in_FT,non Favored Team,Num_SP_in_NFT,NFT_is_ST,ST_in_call
0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,TRAVELING,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,TURNOVER,...,OUT OF BOUNDS,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,FOUL,...,LOOSE BALL,2015,False,NOP,0,0.0,DEN,0.0,0.0,0.0
3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,DOUBLE PERSONAL,2015,False,CLE,1,2.0,HOU,0.0,0.0,1.0
4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,FOUL,...,OFFENSIVE,2015,False,HOU,0,0.0,CLE,2.0,1.0,1.0


In [ ]:
L2M.to_csv('L2M cleaned.csv', encoding = 'utf-8-sig')
files.download('L2M cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>